In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
import pickle

In [ ]:
sales = pd.read_csv('sales.csv')

In [ ]:
#Firstly we check the type of our variables

sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640840 entries, 0 to 640839
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Unnamed: 0           640840 non-null  int64 
 1   store_ID             640840 non-null  int64 
 2   day_of_week          640840 non-null  int64 
 3   date                 640840 non-null  object
 4   nb_customers_on_day  640840 non-null  int64 
 5   open                 640840 non-null  int64 
 6   promotion            640840 non-null  int64 
 7   state_holiday        640840 non-null  object
 8   school_holiday       640840 non-null  int64 
 9   sales                640840 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 48.9+ MB


In [ ]:
#Then we check how our dataframe looks like.

sales.head(5)

,Unnamed: 0,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales
0,425390,366,4,2013-04-18,517,1,0,0,0,4422
1,291687,394,6,2015-04-11,694,1,0,0,0,8297
2,411278,807,4,2013-08-29,970,1,1,0,0,9729
3,664714,802,2,2013-05-28,473,1,1,0,0,6513
4,540835,726,4,2013-10-10,1068,1,1,0,0,10882


In [ ]:
#We rename the column 'Unnamed: 0' for 'index'

sales.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

In [ ]:
#Our date column is the type object, so we transform it to datetime

sales['date'] = pd.to_datetime(sales['date'])

In [ ]:
#We check again that the change was correct

sales.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640840 entries, 0 to 640839
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   index                640840 non-null  int64         
 1   store_ID             640840 non-null  int64         
 2   day_of_week          640840 non-null  int64         
 3   date                 640840 non-null  datetime64[ns]
 4   nb_customers_on_day  640840 non-null  int64         
 5   open                 640840 non-null  int64         
 6   promotion            640840 non-null  int64         
 7   state_holiday        640840 non-null  object        
 8   school_holiday       640840 non-null  int64         
 9   sales                640840 non-null  int64         
dtypes: datetime64[ns](1), int64(8), object(1)
memory usage: 48.9+ MB


In [ ]:
#Now that we have our date column tranform to datetime, we extract the month from this column

sales['month'] = sales['date'].dt.month

#Then we perform one-hot encoding on the month column
month_dummies = pd.get_dummies(sales['month'], prefix='month')

# Concatenate the one-hot encoded columns with the original DataFrame
sales = pd.concat([sales, month_dummies], axis=1)

sales.head(5)

,index,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,425390,366,4,2013-04-18,517,1,0,0,0,4422,...,0,1,0,0,0,0,0,0,0,0
1,291687,394,6,2015-04-11,694,1,0,0,0,8297,...,0,1,0,0,0,0,0,0,0,0
2,411278,807,4,2013-08-29,970,1,1,0,0,9729,...,0,0,0,0,0,1,0,0,0,0
3,664714,802,2,2013-05-28,473,1,1,0,0,6513,...,0,0,1,0,0,0,0,0,0,0
4,540835,726,4,2013-10-10,1068,1,1,0,0,10882,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
#After this we don't need the columns 'date' and 'month' anymore, so we drop then

sales.drop(columns = ['date', 'month'], inplace = True)

In [ ]:
#We check that the change was succesful

sales.head(5)

,index,store_ID,day_of_week,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales,month_1,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,425390,366,4,517,1,0,0,0,4422,0,...,0,1,0,0,0,0,0,0,0,0
1,291687,394,6,694,1,0,0,0,8297,0,...,0,1,0,0,0,0,0,0,0,0
2,411278,807,4,970,1,1,0,0,9729,0,...,0,0,0,0,0,1,0,0,0,0
3,664714,802,2,473,1,1,0,0,6513,0,...,0,0,1,0,0,0,0,0,0,0
4,540835,726,4,1068,1,1,0,0,10882,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
#Now we check the 'state_holiday' column

sales['state_holiday'].unique()

array([0, 1])

In [ ]:
#We change the values [a,b,c] for 1
sales['state_holiday'] = sales['state_holiday'].map({'0':0,'a': 1, 'b': 1,'c':1})


In [ ]:
#Finally, we check for null values

sales.isnull().sum()

index                  0
store_ID               0
day_of_week            0
nb_customers_on_day    0
open                   0
promotion              0
state_holiday          0
school_holiday         0
sales                  0
month_1                0
month_2                0
month_3                0
month_4                0
month_5                0
month_6                0
month_7                0
month_8                0
month_9                0
month_10               0
month_11               0
month_12               0
dtype: int64

In [ ]:
#We order by sales and change the index

store_rank = sales.groupby('store_ID').agg({'sales' : 'sum'}).sort_values(by='sales').reset_index()

In [ ]:
store_rank['new_storeID'] = store_rank['sales'].rank()

In [ ]:
store_rank.drop(columns = ['sales'], inplace=True)

In [ ]:
store_rank.to_csv('store_rank.csv')

In [ ]:
#We merge both df

sales_final = pd.merge(sales, store_rank, on = 'store_ID')

In [ ]:
#We drop the store_Id and index

sales_final.drop(columns = 'index', inplace=True)

In [ ]:
sales_final


,day_of_week,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,new_storeID
0,4,517,1,0,0,0,4422,0,0,0,1,0,0,0,0,0,0,0,0,546.0
1,2,729,1,1,0,0,7880,0,0,1,0,0,0,0,0,0,0,0,0,546.0
2,6,308,1,0,0,0,2922,0,0,0,0,0,0,0,0,0,1,0,0,546.0
3,1,833,1,1,0,0,9821,1,0,0,0,0,0,0,0,0,0,0,0,546.0
4,2,555,1,0,0,1,4772,0,0,0,0,0,0,0,1,0,0,0,0,546.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640835,1,753,1,1,0,0,7007,0,0,0,0,1,0,0,0,0,0,0,0,90.0
640836,7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,90.0
640837,7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,90.0
640838,1,635,1,0,0,0,5706,0,0,0,0,0,0,0,0,0,1,0,0,90.0


In [ ]:
#Split the dataset into features (X) and target variable (y)
X = sales.drop(columns=['sales'])
y = sales['sales']


# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = MinMaxScaler()

X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.fit_transform(X_test)

In [ ]:


# Step 5: Train a machine learning model on the training data
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_normalized, y_train)

# Step 6: Evaluate the model's performance on the testing data
y_pred = model.predict(X_test_normalized)



In [ ]:
#with open('trained_model.pkl', 'wb') as f:
    #pickle.dump(model, f)

pickle.dump(model,open('open_ai.pkl','wb'))

In [142]:
from google.colab import files
files.download('open_ai.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.9087290741407577


In [ ]:
y_pred_final = pd.DataFrame(y_pred, columns=['sales'], index=X_test.index)

final_df = pd.concat([X_test, y_pred_final], axis=1)

In [ ]:
final_df.head(5)

,index,store_ID,day_of_week,nb_customers_on_day,open,promotion,state_holiday,school_holiday,month_1,month_2,...,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,sales
360263,173548,236,3,1081,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,9559.35
226238,645051,856,7,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.00
94660,158198,407,2,346,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,4660.40
322601,188226,942,3,411,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,5233.28
450235,711780,917,2,673,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5543.33


In [ ]:
# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# K-Nearest Neighbors
knn_model = KNeighborsRegressor()
knn_model.fit(X_train, y_train)

# Linear Regression
y_pred_lr = lr_model.predict(X_test)
r2_lr = r2_score(y_test, y_pred_lr)
print("Linear Regression R2 Score:", r2_lr)

# K-Nearest Neighbors
y_pred_knn = knn_model.predict(X_test)
r2_knn = r2_score(y_test, y_pred_knn)
print("K-Nearest Neighbors R2 Score:", r2_knn)

Linear Regression R2 Score: 0.8543890305403068
K-Nearest Neighbors R2 Score: 0.8077630760665612


In [ ]:
feature_importances = model.feature_importances_
feature_names = X.columns

importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
print(importance_df.sort_values(by='Importance', ascending=False))

                Feature  Importance
3   nb_customers_on_day    0.857047
1              store_ID    0.081194
5             promotion    0.023721
0                 index    0.013512
2           day_of_week    0.010378
19             month_12    0.004304
7        school_holiday    0.001494
13              month_6    0.000986
14              month_7    0.000925
12              month_5    0.000865
11              month_4    0.000864
10              month_3    0.000797
8               month_1    0.000749
17             month_10    0.000620
18             month_11    0.000615
9               month_2    0.000603
15              month_8    0.000598
16              month_9    0.000591
6         state_holiday    0.000139
4                  open    0.000000


Running the validation data

In [155]:
sales = pd.read_csv('validation_for students.csv')

store_rank = pd.read_csv('store_rank.csv')

In [162]:
def feature_engineering(sales):

  sales.rename(columns={'Unnamed: 0': 'index'}, inplace=True)


  sales['date'] = pd.to_datetime(sales['date'])


  sales['month'] = sales['date'].dt.month

  #Then we perform one-hot encoding on the month column
  month_dummies = pd.get_dummies(sales['month'], prefix='month')

  # Concatenate the one-hot encoded columns with the original DataFrame
  sales = pd.concat([sales, month_dummies], axis=1)


  sales.drop(columns = ['date', 'month'], inplace = True)


  sales['state_holiday'] = sales['state_holiday'].map({'0':0,'a': 1, 'b': 1,'c':1})




  sales_final = pd.merge(sales, store_rank, on = 'store_ID')


  sales_final.drop(columns = ['index', 'store_ID'],inplace=True)

  return sales

In [163]:
with open('feature_engineering.pkl', 'wb') as f:
    pickle.dump(feature_engineering, f)

In [157]:
y_pred = model.predict(sales_final)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [159]:
predictions = pd.DataFrame(y_pred, index=sales.index)

In [160]:
predictions.to_csv('predictions.csv')

In [164]:
!pip freeze > requirements.txt